In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


This is the main Sensor Kennel application that will 
identify, read and write any sensor that's plugged in.

On the lower side:  
  Pin1 (left) Gnd, Pin3 3v3, Pin-3 (from right) SDA, Pin-5 SCL.
  Pin-2 TX (17), Upper side Pin-2 RX (13) 
  For BNO055 with PS1=High SCL=TX(17), SDA=RX(13)
  
Do we want to draw a graph of numbers if a button is pressed?

Should we have MQTT numbers going out from it?

Or just keep it comprehensive

Sensors known:
* VL53L0X lidar
* VL6180 lidar
* SI7021 humidity
* TSL561 luminous
* TMP102 temp
* SDOF GyAccMag
* BME280 barhumid
* SHT31D tmphumid
* MLX90621 16x4-ir
* PX4PITOT airspeed
* BNO055 orient
* GPS serial


In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [3]:
%sendtofile --binary --source OLED_driver.py
%sendtofile --quiet --source device_detection.py
%sendtofile --quiet --source SI7021_funcs.py
%sendtofile --quiet --source TSL561_funcs.py
%sendtofile --quiet --source VL53L0X_funcs.py
%sendtofile --quiet --source VL6180_funcs.py
%sendtofile --quiet --source SDOF_funcs.py
%sendtofile --quiet --source SHT31D_funcs.py
%sendtofile --quiet --source BME280_funcs.py
%sendtofile --quiet --source MLX90621_funcs.py
%sendtofile --quiet --source BNO055serial_funcs.py
%sendtofile --quiet --source OLED_grapher.py


Sent 3368 bytes in 113 chunks to OLED_driver.py.
Sent 94 lines (2820 bytes) to device_detection.py.
Sent 55 lines (2040 bytes) to SI7021_funcs.py.
Sent 54 lines (1757 bytes) to TSL561_funcs.py.
Sent 33 lines (975 bytes) to VL53L0X_funcs.py.
Sent 106 lines (3190 bytes) to VL6180_funcs.py.
Sent 42 lines (1800 bytes) to SDOF_funcs.py.
Sent 43 lines (1090 bytes) to SHT31D_funcs.py.
Sent 76 lines (3165 bytes) to BME280_funcs.py.
Sent 166 lines (5743 bytes) to MLX90621_funcs.py.
Sent 87 lines (2701 bytes) to BNO055serial_funcs.py.
Sent 145 lines (4083 bytes) to OLED_grapher.py.


In [4]:
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.simple/umqtt/simple.py lib/umqtt/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.robust/umqtt/robust.py lib/umqtt/

Sent 43 lines (1045 bytes) to lib/umqtt/robust.py.


In [5]:
%sendtofile main.py

# connect to the OLED and initialize (and get its i2c bus)
from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext
import machine, time

from device_detection import IdentifyI2CDevice, IdentifyUARTDevice

devices = IdentifyI2CDevice(i2c)
if not devices:
    uart = machine.UART(1, baudrate=9600, rx=13, tx=17)
    # gps is 9600, bluefly is 57600  
    devices = IdentifyUARTDevice(uart)
if not devices:
    import onewire, ds18x20
    pindallas = machine.Pin(13, machine.Pin.OUT, machine.Pin.PULL_UP)
    dallasobj = ds18x20.DS18X20(onewire.OneWire(pindallas))
    dallasscanned = dallasobj.scan()
    if dallasscanned:
        sdallasqorder = {347:0, 376:1, 448:2, 475:3, 576:4, 311:5, 406:6, 157:7}
        dallasscanned.sort(key=lambda X: sdallasqorder.get(sum(X), 0))
        devices = ["dallas n=%d"%len(dallasscanned)]
    
if not devices:
    devices.append("nothing found")

# this might look like a lot of code, but it's the complete 
# visual output of text and numbers that we do per sensor here
for dev in devices:
    devname, devdesc = dev.split(" ", 1)
    fbuff.fill(0)
    fbuff.text(devname, 0, 0, 1)
    fbuff.text(devdesc, 0, 12, 1)
    doublepixels()
    oledshow()
    
# MQTT capabilities (for some devices)
mqttclient = None
if devname in ["SI7021", "dallas"]:
    fbuff.text("Wifi", 0, 56, 1)
    oledshow()
    import network
    si = network.WLAN(network.STA_IF)
    si.active(True)
    if max(s[0]==b'DoESLiverpool'  for s in si.scan()):
        si.connect(b"DoESLiverpool", b"decafbad00")
        from umqtt.robust import MQTTClient
        for i in range(30):
            fbuff.text(".", 32+i*3, 56, 1)
            oledshow()
            if si.isconnected():
                break
            time.sleep(1.0)
        if si.isconnected():
            fbuff.text("********", 32, 56, 1)
            oledshow()
            mqttclient = MQTTClient("sensor_kennel", "10.0.0.2")
            mqttclient.connect()
            time.sleep(1.0)
else:
    time.sleep(2)
    
if devname == "dallas":
    while True:
        dallasobj.convert_temp()
        time.sleep(0.8)
        fbuff.fill(0)
        fbuff.text("dallas temps", 0, 0, 1)
        for i, d in enumerate(dallasscanned):
            try:
                t = dallasobj.read_temp(d)
            except Exception:
                t = -100
            fbuff.text("%.2f"%t, (i%2)*64, (i//2+2)*8)
            if mqttclient is not None and t != -100:
                mqttclient.publish(b"sk1/dallas/%d"%i, b"%.2f"%t)
        if mqttclient:
            fbuff.text("**MQTT", 0, 56)
        oledshow()
    
if devname == "SI7021":
    from SI7021_funcs import setupSI7021, SI7021humiditytemp
    setupSI7021(i2c)
    while True:
        h, t = SI7021humiditytemp()
        fbuff.fill(0)
        fbuff.text("SI7021", 0, 0, 1)
        fbuff.text("humid:", 0, 23, 1)
        fatntext("%.1f"%h, 50, 16)
        fbuff.text("temp:", 0, 53, 1)
        fatntext("%.1f"%t, 50, 46)
        if mqttclient is not None:
            mqttclient.publish(b"sk1/humidity", b"%.1f"%h)
            mqttclient.publish(b"sk1/temperature", b"%.1f"%t)
            fbuff.text("**MQTT", 0, 32)

        oledshow()
        time.sleep(0.25)

if devname == "VL53L0X":
    from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
    VL53L0Xinit(i2c)
    while True:
        d = VL53L0Xdist()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist(mm):", 8, 20, 1)
        fatntext("%d" % d, 40, 36)
        oledshow()
        
if devname == "VL6180":
    from VL6180_funcs import VL6180init, distmm
    VL6180init(i2c)
    while True:
        d = distmm()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist(mm):", 8, 20, 1)
        fatntext("%d" % d, 40, 36)
        oledshow()

if devname == "TSL561":
    from TSL561_funcs import setupTSL561, luminosity, setregtimings
    setupTSL561(i2c)
    setregtimings(True, 1)
    while True:
        br, ir = luminosity()
        fbuff.fill(0)
        fbuff.text("TSL561", 0, 0, 1)
        fbuff.text("broad:", 0, 23, 1)
        fatntext("%d"%br, 50, 16)
        fbuff.text("ir:", 0, 53, 1)
        fatntext("%d"%ir, 50, 46)
        oledshow()
        time.sleep(0.1)

if devname == "TMP102":
    while True:
        r = i2c.readfrom_mem(0x48, 0x00, 2)
        t = r[0] + r[1]/256
        fbuff.fill(0)
        fbuff.text("TMP102", 0, 0, 1)
        fbuff.text("temp:", 0, 23, 1)
        fatntext("%.03f"%t, 30, 40)
        oledshow()
        time.sleep(0.1)

if devname == "SDOF":
    from SDOF_funcs import SetupAccGyrMag, readvectorsensor
    SetupAccGyrMag(i2c)
    while True:
        fbuff.fill(0)
        x, y, z = readvectorsensor("a")
        fbuff.text("Acc:", 0, 0, 1)
        fbuff.text("%d"%x, 8, 8, 1)
        fbuff.text("%d"%y, 8, 16, 1)
        fbuff.text("%d"%z, 8, 24, 1)
        x, y, z = readvectorsensor("g")
        fbuff.text("gyr:", 64, 0, 1)
        fbuff.text("%d"%x, 72, 8, 1)
        fbuff.text("%d"%y, 72, 16, 1)
        fbuff.text("%d"%z, 72, 24, 1)
        x, y, z = readvectorsensor("c")
        fbuff.text("mag:", 40, 32, 1)
        fbuff.text("%d"%x, 48, 40, 1)
        fbuff.text("%d"%y, 48, 48, 1)
        fbuff.text("%d"%z, 48, 56, 1)
        oledshow()
        time.sleep(0.1)
        
if devname == "BME280" or devname == "BME180":
    from BME280_funcs import bme280init, readBME280
    bme280init(i2c)
    while True:
        temp, pressure, humid = readBME280()
        fbuff.fill(0)
        fbuff.fill(0)
        fbuff.text("BME280", 0, 0, 1)
        fbuff.text("tmp:", 0, 16, 1)
        fatntext("%.2f"%(temp), 32, 10)
        fbuff.text("prs:", 0, 36, 1)
        fatntext("%.2f"%(pressure), 32, 30)
        fbuff.text("hum:", 0, 54, 1)
        fatntext("%.2f"%(humid), 32, 50)
        oledshow()
        time.sleep(0.11)

if devname == "SHT31D":
    from SHT31D_funcs import SHT31Dsetup, readSHT31D
    SHT31Dsetup(i2c)
    while True:
        tmp, humid = readSHT31D()
        fbuff.fill(0)
        fbuff.text("SHT31-D", 0, 0, 1)
        fbuff.text("temperature:", 10, 10, 1)
        fatntext("%.2f"%tmp, 20, 22)
        fbuff.text("humidity:", 10, 40, 1)
        fatntext("%.2f"%humid, 20, 50)
        oledshow()
        time.sleep(0.1)
        
if devname == "MLX90621":
    fbuff.text("11", 50, 50, 1)
    oledshow()
    from MLX90621_funcs import measure, cellfillplot
    fbuff.text("22", 59, 50, 1)
    oledshow()
    while True:
        temperatures = measure()
        tmin = min(temperatures)
        tmax = max(tmin+0.001, max(temperatures))
        fbuff.fill(0)
        fbuff.text("MLX90621", 0, 0, 1)
        for i in range(64):
            cellfillplot(fbuff, i//4, 1+(3-(i%4)), ((temperatures[i] - tmin)/(tmax - tmin))**2)
        fbuff.text("min=%.3f"%tmin, 12, 48)
        fbuff.text("max=%.3f"%tmax, 12, 56)
        oledshow()

if devname == "PX4PITOT":
    while True:
        k = i2c.readfrom(0x28, 4)
        status = k[0] & 0xC0; 
        fbuff.fill(0)
        fbuff.text("PX4PITOT", 0, 0, 1)
        if status != 0xC0:
            rawpress = ((k[0] & 0x3F) << 8) | (k[1]) 
            rawtemp = (k[2] << 3) | (k[3] >> 5)
            psi = (rawpress)*(1.0/(0x3FFF*0.4)) - 1.25
            diffpressure = psi*6894.75728
            temp = (rawtemp)*(200.0/0x7FF) - 50
            #airspeed = math.sqrt(abs(2*getdiffpressure())/1.2);
            fbuff.text("diffpress:", 10, 10, 1)
            fatntext("%.2f"%diffpressure, 20, 22)
            fbuff.text("temp:", 10, 40, 1)
            fatntext("%.2f"%temp, 20, 50)
        else:
            fbuff.text("BAD", 20, 20, 1)
        oledshow()
        time.sleep(0.2)
        
if devname == "BNO055":
    from BNO055serial_funcs import InitBNO055, BNO055absorientation, BNO055calibstat
    InitBNO055(uart)
    while True:
        fbuff.fill(0)
        calibstat = None
        try:
            pitch, roll, orient = BNO055absorientation()
            calibstat = BNO055calibstat()
        except Exception as e:
            fbuff.text("BNO055", 0, 0, 1)
            fbuff.text(e.args[0], 0, 16, 1)
        if calibstat is not None:  
            fbuff.text("BNO055 :{0:08b}".format(calibstat), 0, 0, 1)
            fbuff.text("pitch:", 8, 16, 1)
            fatntext("%d"%int(pitch), 64, 10)
            fbuff.text("roll:", 16, 32, 1)
            fatntext("%d"%int(roll), 64, 28)
            fbuff.text("orient:", 0, 52, 1)
            fatntext("%d"%int(orient), 64, 48)
        oledshow()
        time.sleep(0.1)
        
if devname == "GPS":
    while True:
        x = uart.readline()
        if x is not None:
            fbuff.fill(0)
            fbuff.text("GPS serial", 0, 0, 1)
            x = x.strip()
            for i in range(len(x)//16 + 1):
                fbuff.text(x[i*16:i*16+16], 0, 15+i*8, 1)
            oledshow()
            time.sleep_ms(100)
        time.sleep_ms(50)


# read any GPS serial signals on 
fbuff.text("Znone", 120, 56, 1)
oledshow()
time.sleep(2.0)

from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
scrollinit()
a = machine.ADC(machine.Pin(36))  # Pin4 from top left
a.atten(a.ATTN_11DB)
a.width(a.WIDTH_12BIT)
while True:
    v = a.read()
    addscrollgraph(v, time.ticks_ms())
    plotscrollgraph(fbuff)
    fbuff.text("ADCPin36",0,0,1)
    fbuff.text(str(v),0,8,1)
    oledshow()
time.sleep(0.1)
    

Sent 295 lines (9333 bytes) to main.py.


In [14]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [9]:
a = machine.ADC(machine.Pin(36))  # Pin4 from top left
a.atten(a.ATTN_11DB)
a.width(a.WIDTH_12BIT)
scrollinit()
while True:
    v = a.read()
    addscrollgraph(v, time.ticks_ms())
    plotscrollgraph(fbuff)
    fbuff.text("ADCPin36",0,0,1)
    oledshow()


..........................................................
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

